In [12]:
import re
import logging
import boto3
from botocore.exceptions import ClientError
from typing import Any, Text, Dict, List, Optional
from random import randint
from rasa_sdk import Action, Tracker
from rasa_sdk.executor import CollectingDispatcher
from rasa_sdk.events import SlotSet, FollowupAction
WHITELIST_PHONE_NUMBERS_OTP_TESTING = ["+639175330841", "+639154345604"]

logger = logging.getLogger(__name__)

class SNSClient:
    def __init__(self):
        try:
            # Initialize the SNS client
            self.sns_client = boto3.client('sns', region_name='ap-southeast-1')
            logger.info("Successfully initialized SNS client")
        except ClientError as e:
            logger.error(f"Failed to initialize SNS client: {str(e)}")
            raise

    def test_connection(self, test_phone_number: str) -> bool:
        """
        Test SMS sending functionality with a test message.
        Args:
            test_phone_number: Phone number to send test SMS to
        Returns:
            bool: True if successful, False otherwise
        """
        try:
            test_message = "This is a test message from your chatbot."
            result = self.send_sms(test_phone_number, test_message)
            if result:
                logger.info(f"Test SMS sent successfully to {test_phone_number}")
            else:
                logger.error(f"Failed to send test SMS to {test_phone_number}")
            return result
        except Exception as e:
            logger.error(f"Test SMS failed with error: {str(e)}")
            return False

    def send_sms(self, phone_number: str, message: str) -> bool:
        try:
            # Format phone number to E.164 format
            formatted_number = self._format_phone_number(phone_number)
            
            # Check if number is in whitelist
            if formatted_number not in WHITELIST_PHONE_NUMBERS_OTP_TESTING:
                logger.warning(f"Phone number {formatted_number} not in whitelist. SMS not sent.")
                return False
                
            logger.info(f"Sending SMS to whitelisted number: {formatted_number}")
            
            response = self.sns_client.publish(
                PhoneNumber=formatted_number,
                Message=message,
                MessageAttributes={
                    'AWS.SNS.SMS.SMSType': {
                        'DataType': 'String',
                        'StringValue': 'Transactional'
                    }
                }
            )
            logger.info(f"SMS sent successfully: {response['MessageId']}")
            return True
            
        except ClientError as e:
            logger.error(f"Failed to send SMS: {str(e)}")
            return False

    def _format_phone_number(self, phone_number: str) -> str:
        # Remove any non-numeric characters except '+'
        cleaned_number = re.sub(r'[^\d+]', '', phone_number)
        
        # If already in E.164 format (+63XXXXXXXXX), return as is
        if re.match(r'^\+63\d{10}$', cleaned_number):  # Changed from \d{9} to \d{10}
            return cleaned_number
        
        # Handle different formats
        if cleaned_number.startswith('09'):
            formatted_number = '+63' + cleaned_number[1:]
        elif cleaned_number.startswith('63'):
            formatted_number = '+' + cleaned_number
        elif cleaned_number.startswith('0063'):
            formatted_number = '+' + cleaned_number[2:]
        else:
            raise ValueError(f"Invalid phone number format: {phone_number}")
        
        # Final validation
        if not re.match(r'^\+63\d{10}$', formatted_number):  # Changed from \d{9} to \d{10}
            raise ValueError(f"Invalid phone number format - final: {phone_number}, {formatted_number}")
            
        return formatted_number


class ActionInitiateOTPVerification(Action):
    def __init__(self):
        self.sns_client = SNSClient()

    def name(self) -> Text:
        return "action_initiate_otp_verification"

    async def run(
        self, dispatcher: CollectingDispatcher, tracker: Tracker, domain: Dict[Text, Any]
    ) -> List[Dict[Text, Any]]:
        phone_number = tracker.get_slot("user_contact_phone")
        
        if not phone_number or phone_number == 'slot_skipped':
            return []

        # Generate OTP
        otp = randint(100000, 999999)
        
        # Prepare message
        message = f"Your verification code is {otp}. Please enter this code to verify your phone number."
        
        # Send OTP via SNS
        if self.sns_client.send_sms(phone_number, message):
            dispatcher.utter_message(
                text=(
                    "✅ A verification code has been sent to your phone number.\n"
                    "Please enter the 6-digit code to verify your number."
                )
            )
            
            return [
                SlotSet("otp", str(otp)),
                SlotSet("otp_verified", False)
            ]
        else:
            dispatcher.utter_message(
                text=(
                    "❌ Sorry, we couldn't send the verification code.\n"
                    "Would you like to continue without phone verification?"
                ),
                buttons=[
                    {"title": "Try Again", "payload": "/retry_otp"},
                    {"title": "Continue Without Verification", "payload": "/skip_otp_verification"}
                ]
            )
            return []


class ActionVerifyOTP(Action):
    def name(self) -> Text:
        return "action_verify_otp"

    async def run(
        self, dispatcher: CollectingDispatcher, tracker: Tracker, domain: Dict[Text, Any]
    ) -> List[Dict[Text, Any]]:
        user_otp = tracker.latest_message.get("text", "").strip()
        stored_otp = tracker.get_slot("otp")
        
        if not stored_otp:
            return [FollowupAction("action_initiate_otp_verification")]
        
        if user_otp == stored_otp:
            dispatcher.utter_message(text="✅ Phone number verified successfully!")
            return [
                SlotSet("otp_verified", True),
                FollowupAction("contact_form")
            ]
        else:
            dispatcher.utter_message(
                text=(
                    "❌ Invalid verification code.\n"
                    "Please try again or continue without verification."
                ),
                buttons=[
                    {"title": "Try Again", "payload": "/retry_otp"},
                    {"title": "Continue Without Verification", "payload": "/skip_otp_verification"}
                ]
            )
            return [SlotSet("otp_verified", False)]


class ActionSkipOTPVerification(Action):
    def name(self) -> Text:
        return "action_skip_otp_verification"

    async def run(
        self, dispatcher: CollectingDispatcher, tracker: Tracker, domain: Dict[Text, Any]
    ) -> List[Dict[Text, Any]]:
        dispatcher.utter_message(
            text="Continuing without phone verification."
        )
        return [
            SlotSet("otp_verified", False),
            FollowupAction("contact_form")
        ]


In [13]:
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class OTPTester:
    def __init__(self):
        self.sns_client = SNSClient()
        self.current_otp = None
        
    def generate_and_send_otp(self, phone_number: str) -> bool:
        """Generate OTP and send it via SMS"""
        # Generate OTP
        self.current_otp = str(randint(100000, 999999))
        logger.info(f"Generated OTP: {self.current_otp}")
        
        # Prepare message
        message = f"Your verification code is {self.current_otp}. Please enter this code to verify your phone number."
        
        # Send OTP via SNS
        result = self.sns_client.send_sms(phone_number, message)
        if result:
            logger.info(f"OTP sent successfully to {phone_number}")
        else:
            logger.error(f"Failed to send OTP to {phone_number}")
        return result
    
    def verify_otp(self, user_input: str) -> bool:
        """Verify the OTP entered by user"""
        if not self.current_otp:
            logger.error("No OTP has been generated yet")
            return False
            
        is_valid = user_input.strip() == self.current_otp
        logger.info(f"OTP verification {'successful' if is_valid else 'failed'}")
        return is_valid

# Test the OTP flow
def test_otp_flow():
    tester = OTPTester()
    test_number = "09175330841"  # Your test number
    
    # Step 1: Generate and send OTP
    logger.info("Step 1: Generating and sending OTP...")
    if tester.generate_and_send_otp(test_number):
        logger.info("OTP sent successfully")
        
        # Step 2: Simulate user input (for testing, we'll use the actual OTP)
        logger.info("Step 2: Verifying OTP...")
        # In real scenario, you would input the OTP received on phone
        test_input = tester.current_otp  # For testing only
        
        # Verify OTP
        if tester.verify_otp(test_input):
            logger.info("✅ OTP verification successful!")
        else:
            logger.error("❌ OTP verification failed!")
            
        # Test with wrong OTP
        wrong_otp = "111111"
        logger.info(f"Testing with wrong OTP: {wrong_otp}")
        if not tester.verify_otp(wrong_otp):
            logger.info("✅ Wrong OTP correctly rejected!")
    else:
        logger.error("Failed to send OTP")



In [14]:
test_otp_flow()

INFO:__main__:Successfully initialized SNS client
INFO:__main__:Step 1: Generating and sending OTP...
INFO:__main__:Generated OTP: 409624
INFO:__main__:Sending SMS to whitelisted number: +639175330841
INFO:__main__:SMS sent successfully: 0655b198-13ba-5ec3-a39a-d1867dcf83a5
INFO:__main__:OTP sent successfully to 09175330841
INFO:__main__:OTP sent successfully
INFO:__main__:Step 2: Verifying OTP...
INFO:__main__:OTP verification successful
INFO:__main__:✅ OTP verification successful!
INFO:__main__:Testing with wrong OTP: 111111
INFO:__main__:OTP verification failed
INFO:__main__:✅ Wrong OTP correctly rejected!


In [17]:
import logging
from typing import Dict, Any, Text, List
from random import randint

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Mock classes to simulate Rasa components
class MockDispatcher:
    def utter_message(self, text: str, buttons: List[Dict] = None):
        logger.info(f"Bot says: {text}")
        if buttons:
            logger.info(f"Buttons: {buttons}")

class MockTracker:
    def __init__(self):
        self.slots = {}
        self.latest_message = {}
    
    def get_slot(self, slot_name: str) -> Any:
        return self.slots.get(slot_name)
    
    def set_slot(self, slot_name: str, value: Any):
        self.slots[slot_name] = value
        
    def set_user_message(self, message: str):
        self.latest_message = {"text": message}

# Test class for OTP verification
class OTPVerificationTester:
    def __init__(self):
        self.sns_client = SNSClient()
        self.dispatcher = MockDispatcher()
        self.tracker = MockTracker()
    
    def initiate_otp(self, phone_number: str) -> bool:
        """Simulate ActionInitiateOTPVerification"""
        logger.info(f"\nInitiating OTP verification for {phone_number}")
        
        # Store phone number
        self.tracker.set_slot("user_contact_phone", phone_number)
        
        # Generate OTP
        otp = str(randint(100000, 999999))
        logger.info(f"Generated OTP: {otp}")
        
        # Prepare message
        message = f"Your verification code is {otp}. Please enter this code to verify your phone number."
        
        # Send OTP via SNS
        if self.sns_client.send_sms(phone_number, message):
            self.dispatcher.utter_message(
                text="✅ A verification code has been sent to your phone number.\n"
                     "Please enter the 6-digit code to verify your number."
            )
            self.tracker.set_slot("otp", otp)
            self.tracker.set_slot("otp_verified", False)
            return True
        else:
            self.dispatcher.utter_message(
                text="❌ Sorry, we couldn't send the verification code.\n"
                     "Would you like to continue without phone verification?",
                buttons=[
                    {"title": "Try Again", "payload": "/retry_otp"},
                    {"title": "Continue Without Verification", "payload": "/skip_otp_verification"}
                ]
            )
            return False
    
    def verify_otp(self, user_input: str) -> bool:
        """Simulate ActionVerifyOTP"""
        logger.info(f"\nVerifying OTP: {user_input}")
        
        # Set user input
        self.tracker.set_user_message(user_input)
        
        # Get stored OTP
        stored_otp = self.tracker.get_slot("otp")
        logger.info(f"Stored OTP: {stored_otp}")
        
        if not stored_otp:
            logger.warning("No stored OTP found!")
            return False
        
        # Verify OTP
        if user_input.strip() == stored_otp:
            logger.info("OTP verification successful!")
            self.dispatcher.utter_message(text="✅ Phone number verified successfully!")
            self.tracker.set_slot("otp_verified", True)
            return True
        else:
            logger.warning("OTP verification failed!")
            self.dispatcher.utter_message(
                text="❌ Invalid verification code.\n"
                     "Please try again or continue without verification.",
                buttons=[
                    {"title": "Try Again", "payload": "/retry_otp"},
                    {"title": "Continue Without Verification", "payload": "/skip_otp_verification"}
                ]
            )
            self.tracker.set_slot("otp_verified", False)
            return False

def test_otp_verification_flow():
    tester = OTPVerificationTester()
    test_number = "09175330841"  # Your test number
    
    # Test 1: Initiate OTP
    logger.info("\n=== Test 1: Initiating OTP ===")
    if tester.initiate_otp(test_number):
        stored_otp = tester.tracker.get_slot("otp")
        
        # Test 2: Verify with correct OTP
        logger.info("\n=== Test 2: Verifying with correct OTP ===")
        tester.verify_otp(stored_otp)
        
        # Test 3: Verify with wrong OTP
        logger.info("\n=== Test 3: Verifying with wrong OTP ===")
        tester.verify_otp("111111")
    
    # Print final state
    logger.info("\n=== Final State ===")
    logger.info(f"Slots: {tester.tracker.slots}")

def run_test_cases():
    logger.info("\n=== Starting OTP Verification Test Cases ===")
    tester = OTPVerificationTester()
    
    # Test Case 1: Happy Path - Correct OTP
    logger.info("\n🧪 Test Case 1: Happy Path - Correct OTP")
    test_number = "09175330841"
    if tester.initiate_otp(test_number):
        stored_otp = tester.tracker.get_slot("otp")
        assert tester.verify_otp(stored_otp), "Should verify with correct OTP"
        assert tester.tracker.get_slot("otp_verified") is True, "otp_verified should be True"
    
    # Test Case 2: Wrong OTP
    logger.info("\n🧪 Test Case 2: Wrong OTP")
    tester = OTPVerificationTester()  # Reset tester
    if tester.initiate_otp(test_number):
        assert not tester.verify_otp("111111"), "Should fail with wrong OTP"
        assert tester.tracker.get_slot("otp_verified") is False, "otp_verified should be False"
    
    # Test Case 3: Multiple Wrong Attempts
    logger.info("\n🧪 Test Case 3: Multiple Wrong Attempts")
    tester = OTPVerificationTester()
    if tester.initiate_otp(test_number):
        stored_otp = tester.tracker.get_slot("otp")
        wrong_attempts = ["111111", "222222", "333333"]
        for attempt in wrong_attempts:
            assert not tester.verify_otp(attempt), f

In [20]:
# Cell 2: Function to initiate OTP
def send_otp(phone_number: str):
    global tester
    tester = OTPVerificationTester()
    success = tester.initiate_otp(phone_number)
    if success:
        logger.info("OTP sent successfully! Check your phone.")
        logger.info(f"Current slots: {tester.tracker.slots}")
    return success



# Cell 3: Function to verify OTP
def verify_received_otp(user_input: str):
    global tester
    if not tester:
        logger.error("No active OTP session. Please send OTP first.")
        return False
        
    success = tester.verify_otp(user_input)
    logger.info(f"Verification {'successful' if success else 'failed'}!")
    logger.info(f"Current slots: {tester.tracker.slots}")
    return success

In [21]:
# Run this to send OTP
phone_number = "09175330841"  # Your test number
send_otp(phone_number)

INFO:__main__:Successfully initialized SNS client
INFO:__main__:
Initiating OTP verification for 09175330841
INFO:__main__:Generated OTP: 306878
INFO:__main__:Sending SMS to whitelisted number: +639175330841
INFO:__main__:SMS sent successfully: afdd5547-2a15-57ed-9e7f-35595a034cf5
INFO:__main__:Bot says: ✅ A verification code has been sent to your phone number.
Please enter the 6-digit code to verify your number.
INFO:__main__:OTP sent successfully! Check your phone.
INFO:__main__:Current slots: {'user_contact_phone': '09175330841', 'otp': '306878', 'otp_verified': False}


True

In [23]:
verify_received_otp("306878")

INFO:__main__:
Verifying OTP: 306878
INFO:__main__:Stored OTP: 306878
INFO:__main__:OTP verification successful!
INFO:__main__:Bot says: ✅ Phone number verified successfully!
INFO:__main__:Verification successful!
INFO:__main__:Current slots: {'user_contact_phone': '09175330841', 'otp': '306878', 'otp_verified': True}


True